In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%reset -sf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from pprint import pprint
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from itertools import repeat, combinations
from json import dumps, loads
from time import monotonic
from tqdm import tqdm

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np

import matplotlib.pyplot as plt

#!pip install -U scikit-learn
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV, LarsCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

#from torch import nn, rand, as_tensor, from_numpy, no_grad, sigmoid, column_stack
#from torch.utils.data import Dataset, DataLoader
#import torch.optim as pt_optim
#from torch.cuda import is_available

#from optuna import create_study, exceptions

In [ ]:
root = Path('/kaggle/input/tabular-playground-series-nov-2021/')
root

from sys import getsizeof
for file in root.glob('*'):
    print(file.stem, end=': ')
    with open(file, 'r') as file:
        print(f'{getsizeof(file.read())/1000000:,.2f} Mb')

In [ ]:
USE_SAMPLE = True  # True for fast dev results

train = pd.read_csv(root / 'train.csv', low_memory=False)
test = pd.read_csv(root / 'test.csv', low_memory=False)

if USE_SAMPLE:
    train = train.sample(frac=0.1)
    test = test.sample(frac=0.1)

train_X = train.drop(['id', 'target'], axis=1)
train_X

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.cluster import MiniBatchKMeans

# Main Pipe

rb = RobustScaler()
pca = PCA(n_components=20)
km = MiniBatchKMeans(n_clusters=2)

pipe = make_pipeline(rb, pca, km)
pipe.fit(train_X)

pipe['pca'].explained_variance_ratio_.sum()

train_X_ = pipe['pca'].transform(train_X)

In [ ]:
# Real labels vs estimated labels

train['target'].value_counts()
pd.DataFrame(pipe.predict(train_X)).value_counts()

#### Why such a difference?
It'd be expected a similar structure of true labels vs estimated (by X) labels
This suggests that similar samples have mixed labels
Which in turn suggests missing but needed data -original or additional-, or,
in the worst case, incorrect labeling...

In [ ]:
# Visualize real vs estimated labels

pairs = zip(
    range(pipe['pca'].n_components-1), 
    range(1, pipe['pca'].n_components))

for i, (col, col_) in enumerate(pairs):
    fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 7), sharex=True, sharey=True)
    _ = ax[0].scatter(train_X_[:, col], train_X_[:, col_], label='True', c=train['target'], s=15, alpha=0.5)
    _ = ax[1].scatter(train_X_[:, col], train_X_[:, col_], label='Estimated', c=pipe.predict(train_X), s=15, alpha=0.5)
    _ = ax[0].legend()
    _ = ax[1].legend()
    plt.show()
    plt.close()